In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

# model

In [ ]:
class RNN:
    
    def __init__(self):
        self._create_placeholder()
        self._create_model()
    
    def _create_placeholder(self):
        pass
    
    def _create_model(self):
        pass
    
    def _bidirectional_cell(self):
        pass
    
    def _gru(self):
        pass
    